## Download and open the text

In [1]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters:", len(raw_text))
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


## Build vocabulary with sorted unique tokens from the text

In [2]:
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
all_words = sorted(set(preprocessed))
all_words.extend(["<|endoftext|>", "<|unk|>"])
vocab_size = len(all_words)
vocab = {token:integer for integer,token in enumerate(all_words)}

print(len(preprocessed))
print(vocab_size)

4690
1132


## A simple Tokenizer with two special tokens <|unk|> and <|endoftext|>

In [3]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        tokens = [self.int_to_str[i] for i in ids]
        text = " ".join(tokens)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


## Example of using the Tokenizer

In [63]:
tokenizer = SimpleTokenizerV1(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print("Text: ",text)

ids = tokenizer.encode(text)
print("Encoded ids: ", ids)

decoded_text = tokenizer.decode(ids)
print("Decoded text: ", decoded_text)


Text:  Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
Encoded ids:  [1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
Decoded text:  <|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


## Byte Pair Encoding

In [64]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [65]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [66]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [67]:
text = "Akwirw ier"

ids = tokenizer.encode(text)
print(ids)

str = tokenizer.decode(ids)
print(str)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


In [68]:
enc_ids = tokenizer.encode(raw_text)
print(len(enc_ids))

5145


## Dataloader via Pytorch

In [69]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)
        tokens_len = len(token_ids)

        for i in range(0, tokens_len - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1: i+max_length+1]

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride,)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
    return dataloader

In [70]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

max_length=4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


## Attentioin
### Simplified self-attention

In [71]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

# E.g., calculate context vector of x^2, journey
query = inputs[1]  # x^2
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print("attention scores for x^2:\n", attn_scores_2, "\n")

att_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("attention weights for x^2:\n", att_weights_2, "\n")
print("sum: ", att_weights_2.sum(), "\n")

context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += att_weights_2[i] * x_i
print("Context vector for x^2:\n", context_vec_2, "\n")

# Now calculate context vectors of all embeddings
att_scores = inputs @ inputs.T
att_weights = torch.softmax(att_scores, dim=-1)
context_vec = att_weights @ inputs
print("Context vector:\n", context_vec, "\n")


attention scores for x^2:
 tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865]) 

attention weights for x^2:
 tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581]) 

sum:  tensor(1.) 

Context vector for x^2:
 tensor([0.4419, 0.6515, 0.5683]) 

Context vector:
 tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]]) 



### Self-attention with trainable weights

In [72]:
# Example using x^2

x_2 = inputs[1]
d_in = inputs.shape[1]
print("input embedding shape: ", inputs.shape, "\n")
d_out = 2

torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print("query_2: ", query_2, "\n")
print("key 2: ", key_2, "\n")
keys = inputs @ W_key
print("keys[1]: ", keys[1], "\n")
values = inputs @ W_value

att_score_22 = query_2.dot(key_2)

att_scores_2 = query_2 @ keys.T

d_k = keys.shape[-1]
print("d_k: ", d_k, "\n")
att_weights_2 = torch.softmax(att_scores_2 / d_k ** 0.5, dim=-1)
print("att scores 2:\n", att_scores_2, "\n")
print("att weights 2:\n", att_weights_2, "\n")
print("sum: ", att_weights_2.sum(), "\n")

context_vec_2 = att_weights_2 @ values
print("context vector 2:\n", context_vec_2, "\n")

input embedding shape:  torch.Size([6, 3]) 

query_2:  tensor([0.4306, 1.4551]) 

key 2:  tensor([0.4433, 1.1419]) 

keys[1]:  tensor([0.4433, 1.1419]) 

d_k:  2 

att scores 2:
 tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440]) 

att weights 2:
 tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820]) 

sum:  tensor(1.0000) 

context vector 2:
 tensor([0.3061, 0.8210]) 



In [73]:
import torch.nn as nn

class SelfAttentionV1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()

        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_keys = nn.Parameter(torch.rand(d_in, d_out))
        self.W_values = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        queries = x @ self.W_query # x_0 * d_out
        keys = x @ self.W_keys # x_0 * d_out
        values = x @ self.W_values # x_0 * d_out

        att_scores = queries @ keys.T # x_0 * x_0
        att_weights = torch.softmax(att_scores / d_out ** 0.5, dim=-1) # x_0 * x_0
        context_vec = att_weights @ values # x_0 * d_out
        return context_vec

torch.manual_seed(123)
self_att_v1 = SelfAttentionV1(d_in, d_out)
print("context vector:\n", self_att_v1.forward(inputs), "\n")


class SelfAttentionV2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        queries = self.W_query(x) # x_0 * d_out
        keys = self.W_key(x) # x_0 * d_out
        values = self.W_value(x) # x_0 * d_out

        att_scores = queries @ keys.T # x_0 * x_0
        att_weights = torch.softmax(att_scores / keys.shape[-1] ** 0.5, dim=-1) # x_0 * x_0
        context_vec = att_weights @ values # x_0 * d_out
        return context_vec

torch.manual_seed(789)
self_att_v2 = SelfAttentionV2(d_in, d_out)
print("context vector:\n", self_att_v2.forward(inputs), "\n")

context vector:
 tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>) 

context vector:
 tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>) 



### Casual Attention

In [74]:
class CasualAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()

        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length),diagonal=1)
        )

    def forward(self, x):
        batch_count, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        att_scores = queries @ keys.transpose(1, 2)
        att_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)

        att_weights = torch.softmax(att_scores / keys.shape[-1] ** 0.5, dim=-1)
        att_weights = self.dropout(att_weights)

        context_vec = att_weights @ values
        return context_vec

batch = torch.stack((inputs, inputs), dim=0)
print("batch: ", batch.shape)

torch.manual_seed(123)
context_length = batch.shape[1]

casual_att = CasualAttention(d_in, d_out, context_length, 0.5)
context_vecs = casual_att.forward(batch)
print("context vectors shape:", context_vecs.shape, "\n")
print("context vectors:\n", context_vecs, "\n")


batch:  torch.Size([2, 6, 3])
context vectors shape: torch.Size([2, 6, 2]) 

context vectors:
 tensor([[[-0.9038,  0.4432],
         [-0.4368,  0.2142],
         [-0.4849, -0.1341],
         [-0.5834,  0.0081],
         [-0.6219, -0.0526],
         [-0.1417, -0.0505]],

        [[ 0.0000,  0.0000],
         [-1.1749,  0.0116],
         [-0.7733,  0.0073],
         [-0.9140, -0.2769],
         [-0.7679, -0.0735],
         [-0.6749, -0.0984]]], grad_fn=<UnsafeViewBackward0>) 



### Multi-head attention

In [75]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # dimensions in each head
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x): # x.shape = [batches, num_tokens, d_in], d_in = embedding dimentions
        batches, num_tokens, d_in = x.shape

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(batches, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(batches, num_tokens, self.num_heads, self.head_dim)
        values = values.view(batches, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)      # (batches, num_heads, num_tokens, head_dim)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        att_scores = queries @ keys.transpose(-2, -1)

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        att_scores.masked_fill_(mask_bool, -torch.inf)
        att_weights = torch.softmax(att_scores / keys.shape[-1] ** 0.5, dim=-1)

        att_weights = self.dropout(att_weights)

        context_vec = (att_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(batches, num_tokens, d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec

In [76]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
